In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re

import seaborn as sns
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model

print(os.listdir("../input"))

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', -1)

# Any results you write to the current directory are saved as output.

Using TensorFlow backend.


['hindienglish-corpora', 'test-2']


In [2]:
lines=pd.read_csv("../input/test-2/eng-asm.csv",encoding='utf-8')

In [3]:
lines.head(20)

,english_sentence,assamese_sentence
0,Hi.,নমস্কাৰ।
1,Hi.,হাই।
2,Run!,দৌৰ!
3,Run!,দৌৰাঁ!
4,Run!,দৌৰক!
5,Who?,কোন?
6,Wow!,বাঃ!
7,Fire!,জুই!
8,Help!,সহায় কৰাঁ!
9,Help!,সহায় কৰক!


In [4]:
pd.isnull(lines).sum()

english_sentence     0
assamese_sentence    0
dtype: int64

In [5]:
lines.drop_duplicates(inplace=True)

* ### Let us pick any 25000 rows from the dataset.

In [6]:
# Lowercase all characters
lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.lower())
lines['assamese_sentence']=lines['assamese_sentence'].apply(lambda x: x.lower())

In [7]:
lines.shape

(1745, 2)

In [8]:
# Remove quotes
lines['english_sentence']=lines['english_sentence'].apply(lambda x: re.sub("'", '', x))
#lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: re.sub("'", '', x))

In [9]:
exclude = set(string.punctuation)
print(exclude)

{'`', '/', '&', '~', '}', '_', ';', '?', '<', '|', ':', '^', '#', "'", '[', '=', '(', '{', '-', '!', '\\', '+', '$', '"', '*', ',', '>', '%', '@', ']', '.', ')'}


In [10]:
exclude = set(string.punctuation) # Set of all special characters
# Remove all the special characters
lines['english_sentence']=lines['english_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
#lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [11]:
# Remove all numbers from text
remove_digits = str.maketrans('', '', digits)
lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.translate(remove_digits))
lines['assamese_sentence']=lines['assamese_sentence'].apply(lambda x: x.translate(remove_digits))

lines['assamese_sentence'] = lines['assamese_sentence'].apply(lambda x: re.sub("[০১২৩৪৫৬৭৮৯]", "", x))

# Remove extra spaces
lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.strip())
lines['assamese_sentence']=lines['assamese_sentence'].apply(lambda x: x.strip())
lines['english_sentence']=lines['english_sentence'].apply(lambda x: re.sub(" +", " ", x))
lines['assamese_sentence']=lines['assamese_sentence'].apply(lambda x: re.sub(" +", " ", x))


In [12]:
# Add start and end tokens to target sequences
lines['assamese_sentence'] = lines['assamese_sentence'].apply(lambda x : 'START_ '+ x + ' _END')

In [13]:
lines.tail()

,english_sentence,assamese_sentence
1740,its so hot that you could cook an egg on the hood of a car,START_ ইমানেই গৰম যে আপুনি গাড়ীৰ হুডত কণী সিজাব পাৰিব। _END
1741,its so hot that you could cook an egg on the hood of a car,START_ ইমানেই গৰম যে তুমি গাড়ীৰ হুডত কণী সিজাব পাৰিবা। _END
1742,i just kept asking tom the same question but he never answered me,START_ মই টমক একেই প্ৰশ্ন সুধিয়েই থাকোঁ কিন্তু সি মোক কেতিয়াও উত্তৰ দিয়া নাই। _END
1743,it took me more than two hours to translate a few pages of english,START_ ইংৰাজীৰ কেইখনমান পৃষ্ঠা অনুবাদ কৰোঁতে মোৰ দুঘণ্টাৰো অধিক সময় লাগিল। _END
1744,tom mentioned that he and mary were thinking about getting married,START_ টম্‌ আৰু মেৰীয়ে বিয়া পতাৰ কথা ভাবি আছে বুলি তেওঁ উল্লেখ কৰিলে। _END


In [14]:
### Get English and Hindi Vocabulary
all_eng_words=set()
for eng in lines['english_sentence']:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

all_assamese_words=set()
for assm in lines['assamese_sentence']:
    for word in assm.split():
        if word not in all_assamese_words:
            all_assamese_words.add(word)

In [15]:
len(all_eng_words)

1218

In [16]:
len(all_assamese_words)

2192

In [17]:
lines['length_eng_sentence']=lines['english_sentence'].apply(lambda x:len(x.split(" ")))
lines['length_assm_sentence']=lines['assamese_sentence'].apply(lambda x:len(x.split(" ")))

In [18]:
lines.head()

,english_sentence,assamese_sentence,length_eng_sentence,length_assm_sentence
0,hi,START_ নমস্কাৰ। _END,1,3
1,hi,START_ হাই। _END,1,3
2,run,START_ দৌৰ! _END,1,3
3,run,START_ দৌৰাঁ! _END,1,3
4,run,START_ দৌৰক! _END,1,3


In [19]:
lines[lines['length_eng_sentence']>30].shape

(0, 4)

In [20]:
lines=lines[lines['length_eng_sentence']<=20]
lines=lines[lines['length_assm_sentence']<=20]

In [21]:
lines.shape

(1745, 4)

In [22]:
print("maximum length of Assamese Sentence ",max(lines['length_assm_sentence']))
print("maximum length of English Sentence ",max(lines['length_eng_sentence']))

maximum length of Assamese Sentence  16
maximum length of English Sentence  15


In [23]:
max_length_src=max(lines['length_assm_sentence'])
max_length_tar=max(lines['length_eng_sentence'])

In [24]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_assamese_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_assamese_words)
num_encoder_tokens, num_decoder_tokens

(1218, 2192)

In [25]:
num_decoder_tokens += 1 #for zero padding

In [26]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [27]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [28]:
reverse_input_char_index

{1: 'a',
 2: 'aah',
 3: 'able',
 4: 'aboard',
 5: 'about',
 6: 'above',
 7: 'abroad',
 8: 'absent',
 9: 'accept',
 10: 'accepted',
 11: 'accident',
 12: 'accustomed',
 13: 'act',
 14: 'acting',
 15: 'active',
 16: 'add',
 17: 'address',
 18: 'adjust',
 19: 'admissions',
 20: 'admitted',
 21: 'advice',
 22: 'africa',
 23: 'after',
 24: 'afternoon',
 25: 'again',
 26: 'against',
 27: 'ahead',
 28: 'air',
 29: 'airmail',
 30: 'alarm',
 31: 'alive',
 32: 'all',
 33: 'allergic',
 34: 'allowed',
 35: 'almost',
 36: 'alone',
 37: 'already',
 38: 'also',
 39: 'always',
 40: 'am',
 41: 'among',
 42: 'amused',
 43: 'an',
 44: 'and',
 45: 'angry',
 46: 'answer',
 47: 'answered',
 48: 'any',
 49: 'anybody',
 50: 'anymore',
 51: 'anyone',
 52: 'anything',
 53: 'apparent',
 54: 'applauded',
 55: 'apple',
 56: 'apples',
 57: 'apron',
 58: 'are',
 59: 'arent',
 60: 'argue',
 61: 'arm',
 62: 'around',
 63: 'arrested',
 64: 'arrival',
 65: 'arrive',
 66: 'arrived',
 67: 'arrives',
 68: 'artist',
 69: 'a

In [29]:
lines.head(10)

,english_sentence,assamese_sentence,length_eng_sentence,length_assm_sentence
0,hi,START_ নমস্কাৰ। _END,1,3
1,hi,START_ হাই। _END,1,3
2,run,START_ দৌৰ! _END,1,3
3,run,START_ দৌৰাঁ! _END,1,3
4,run,START_ দৌৰক! _END,1,3
5,who,START_ কোন? _END,1,3
6,wow,START_ বাঃ! _END,1,3
7,fire,START_ জুই! _END,1,3
8,help,START_ সহায় কৰাঁ! _END,1,4
9,help,START_ সহায় কৰক! _END,1,4


In [30]:
lines = shuffle(lines)
lines.head(10)

,english_sentence,assamese_sentence,length_eng_sentence,length_assm_sentence
1388,it was great to see you again,START_ আপোনাক আকৌ লগ পাই বৰ ভাল লাগিল। _END,7,9
986,my father gave me a game,START_ মোৰ দেউতাই মোক এটা গেম দিলে। _END,6,8
1644,thats the reason i accepted the offer,START_ সেই কাৰণেই মই অফাৰটো গ্ৰহণ কৰিলোঁ। _END,7,8
250,you are hopeless,START_ আপুনি অলায়ক। _END,3,4
558,i count on your help,START_ মই তোমাৰ সহায়ৰ ওপৰত নিৰ্ভৰশীল। _END,5,7
392,was the movie good,START_ কথাছবিখন ভাল আছিলনে? _END,4,5
225,please elaborate,START_ বহলাই কোৱাচোন। _END,2,4
1671,i really dont want to live in australia,START_ মই সঁচাকৈয়ে অষ্ট্ৰেলিয়াত থাকিব নিবিচাৰোঁ। _END,8,7
1516,please take off your muddy boots,START_ আপোনাৰ বোকা লাগি থকা বুটযোৰ খুলি থওকচোন। _END,6,9
1702,the scenery was beautiful beyond description,START_ বৰ্ণনা কৰিব নোৱাৰা সুন্দৰ দৃশ্য আছিল। _END,6,8


### Split the data into train and test

In [31]:
X, y = lines['english_sentence'], lines['assamese_sentence']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=42)
X_train.shape, X_test.shape

((1396,), (349,))

### Let us save this data

In [32]:
X_train.to_pickle('X_train.pkl')
X_test.to_pickle('X_test.pkl')


In [33]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

### Encoder-Decoder Architecture

In [34]:
latent_dim=300

In [35]:
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

Instructions for updating:
Colocations handled automatically by placer.


In [36]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [37]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [38]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 300)    365400      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 300)    657900      input_2[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LS

In [39]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 128
epochs = 100

In [40]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)



Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/100
10/10 [==============================] - 5s 479ms/step - loss: 6.8822 - val_loss: 6.1382
Epoch 2/100
10/10 [==============================] - 1s 70ms/step - loss: 5.8079 - val_loss: 5.9710
Epoch 3/100
10/10 [==============================] - 1s 67ms/step - loss: 5.5580 - val_loss: 5.8350
Epoch 4/100
10/10 [==============================] - 1s 60ms/step - loss: 5.4072 - val_loss: 5.7364
Epoch 5/100
10/10 [==============================] - 1s 60ms/step - loss: 5.2769 - val_loss: 5.8005
Epoch 6/100
10/10 [==============================] - 1s 60ms/step - loss: 5.1544 - val_loss: 5.6554
Epoch 7/100
10/10 [==============================] - 1s 61ms/step - loss: 5.0400 - val_loss: 5.6666
Epoch 8/100
10/10 [==============================] - 1s 61ms/step - loss: 4.9290 - val_loss: 5.7121
Epoch 9/100
10/10 [==============================] - 1s 61ms/step - loss:

Epoch 78/100
10/10 [==============================] - 1s 59ms/step - loss: 0.3363 - val_loss: 5.2668
Epoch 79/100
10/10 [==============================] - 1s 60ms/step - loss: 0.3233 - val_loss: 5.1108
Epoch 80/100
10/10 [==============================] - 1s 58ms/step - loss: 0.3058 - val_loss: 5.2964
Epoch 81/100
10/10 [==============================] - 1s 59ms/step - loss: 0.2931 - val_loss: 5.3059
Epoch 82/100
10/10 [==============================] - 1s 58ms/step - loss: 0.2791 - val_loss: 5.1312
Epoch 83/100
10/10 [==============================] - 1s 81ms/step - loss: 0.2765 - val_loss: 5.3377
Epoch 84/100
10/10 [==============================] - 1s 63ms/step - loss: 0.2609 - val_loss: 5.3840
Epoch 85/100
10/10 [==============================] - 1s 58ms/step - loss: 0.2534 - val_loss: 5.1711
Epoch 86/100
10/10 [==============================] - 1s 97ms/step - loss: 0.2385 - val_loss: 5.3549
Epoch 87/100
10/10 [==============================] - 1s 65ms/step - loss: 0.2336 - val_los

In [ ]:
# model.save_weights('nmt_weights.h5')

In [41]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)


In [42]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [46]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
for k in range(-1,20):
    k+=1
    (input_seq, actual_output), _ = next(train_gen)
    decoded_sentence = decode_sequence(input_seq)
    print('Input English sentence:', X_train[k:k+1].values[0])
    print('Actual Assamese Translation:', y_train[k:k+1].values[0][6:-4])
    print('Predicted Assamese Translation:', decoded_sentence[:-4])
    print('*************************\n')

Input English sentence: ive been arrested several times
Actual Assamese Translation:  মোক কেইবাবাৰো গ্ৰেপ্তাৰ কৰা হৈছে। 
Predicted Assamese Translation:  মোক কেইবাবাৰো গ্ৰেপ্তাৰ কৰা হৈছে। 
*************************

Input English sentence: he blamed me for not coming
Actual Assamese Translation:  তেওঁ নহাৰ বাবে মোক দোষ দিছে। 
Predicted Assamese Translation:  তেওঁ নহাৰ বাবে মোক দোষ দিছে। 
*************************

Input English sentence: it doesnt snow here very often
Actual Assamese Translation:  ইয়াত ইমান সঘনাই তুষাৰপাত নহয়। 
Predicted Assamese Translation:  ইয়াত ইমান ঘনাই বৰফ নপৰে। 
*************************

Input English sentence: how are you
Actual Assamese Translation:  তুমি কেনে আছা? 
Predicted Assamese Translation:  আপুনি কেনে আছে? 
*************************

Input English sentence: no one doubts that tom will do that tomorrow
Actual Assamese Translation:  নিঃসন্দেহে টমে সেয়া কাইলৈ কৰিব। 
Predicted Assamese Translation:  নিঃসন্দেহে টমে সেয়া কাইলৈ কৰিব। 
******************

In [55]:
test_gen = generate_batch(X_test, y_test, batch_size = 1)
for k in range(-1,20):
    k+=1
    (input_seq, actual_output), _ = next(test_gen)
    decoded_sentence = decode_sequence(input_seq)
    print('Input English sentence:', X_test[k:k+1].values[0])
    print('Actual Assamese Translation:', y_test[k:k+1].values[0][6:-4])
    print('Predicted Assamese Translation:', decoded_sentence[:-4])
    print('*************************\n')

Input English sentence: we had a lot of snow last year
Actual Assamese Translation:  যোৱাবছৰ খুব বৰফ পৰিছিল। 
Predicted Assamese Translation:  যোৱাবছৰ খুব তুষাৰপাত হৈছিল। 
*************************

Input English sentence: buy some milk on your way home
Actual Assamese Translation:  ঘৰলৈ আহোঁতে অলপ গাখীৰ কিনি আনিবা। 
Predicted Assamese Translation:  ঘৰলৈ আহোঁতে অলপ গাখীৰ কিনি আনিব। 
*************************

Input English sentence: my father was busy
Actual Assamese Translation:  মোৰ দেউতা ব্যস্ত আছিল। 
Predicted Assamese Translation:  মোৰ দেউতা ব্যস্ত। 
*************************

Input English sentence: what is your name
Actual Assamese Translation:  তোমাৰ নামটো কি? 
Predicted Assamese Translation:  আপোনাৰ নাম কি? 
*************************

Input English sentence: it seems interesting to me
Actual Assamese Translation:  মোৰ আমোদজনক যেন লাগিছে। 
Predicted Assamese Translation:  তুমি বৰ খোজানে? 
*************************

Input English sentence: i made my son a new suit
Actual Assames